In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd 
import re
import copy
from time import sleep

DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = 'localhost'
USER = 'postgres'
PASSWORD = 'Taj290898'
DATABASE = 'Pagila'
PORT = 5432

In [2]:
with psycopg2.connect(host=HOST, user=USER, password=PASSWORD, dbname=DATABASE, port=PORT) as conn:
    with conn.cursor() as cur:
        cur.execute('''SELECT * from FILM 
                        LIMIT 10''')
        records = cur.fetchall()

In [3]:
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")


In [4]:
actors = pd.read_sql_table('film', engine)

title_and_release_year = pd.read_sql_query('''SELECT title as Title, release_year as Year from film''', engine)

title_and_release_year.head(5)

,title,year
0,ACADEMY DINOSAUR,2006
1,ACE GOLDFINGER,2006
2,ADAPTATION HOLES,2006
3,AFFAIR PREJUDICE,2006
4,AFRICAN EGG,2006


In [5]:
slj_xpath = ['//div[@class="film-row odd"]', '//div[@class="filmo-row even"]']

class ImdbScraper:
    def __init__(self):
        self.driver = webdriver.Safari()
        self.engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
    
    def get_data(self, url):
        try:

            self.driver.get(url)
            sleep(5)
            information = pd.DataFrame()
            for xpath in slj_xpath:
                div_tags = self.driver.find_elements(By.XPATH, xpath)
                for tag in div_tags:
                    film_name = tag.find_element_by_xpath('./b/a').text
                    year = tag.find_element_by_xpath('./span').text
                    information = information.append({'film_name': film_name, 'year': year}, ignore_index=True)

            return information
        except: 
            raise ValueError('URL Not Valid')
    
        
    


In [20]:
scraper = ImdbScraper()

tr_data = scraper.get_data('https://www.imdb.com/name/nm0000619/?ref_=nv_sr_srsg_0')

slj_data = scraper.get_data('https://www.imdb.com/name/nm0000168/?ref_=nv_sr_srsg_0')

scraper.driver.quit()


In [22]:
c = '2018'
new_pattern = re.compile(r'[0-9]+(-[0-9]+)?')
j = new_pattern.search(c)[0]
print(c)
print(j)

2018
2018


,film_name,year
0,She-Hulk,\n 2022\n
1,Sundown,\n 2021\n
2,Bergman Island,\n 2021\n
3,Tin Star,\n 2017-2020\n
4,Once Upon a Time... In Hollywood,\n 2019\n


In [23]:

pattern = re.compile(r'[0-9]+(-[0-9]+)?')
def _year_finder(string):
    try:
        year_val = pattern.search(string)[0]
        return year_val
    except:
        pass 
tr_data['year'] = tr_data['year'].apply(lambda year: year_finder(year))
slj_data['year'] = slj_data['year'].apply(lambda year: year_finder(year))

In [30]:
tr_data.head()

,film_name,year
0,She-Hulk,2022
1,Sundown,2021
2,Bergman Island,2021
3,Tin Star,2017-2020
4,Once Upon a Time... In Hollywood,2019


In [25]:
tr_data.to_sql('tim_roth_movies', engine, if_exists='replace')
slj_data.to_sql('samuel_l_jackson_movies',engine, if_exists='replace')


In [26]:
engine.execute(''' SELECT tim_roth_movies.film_name, tim_roth_movies.year 
                   FROM tim_roth_movies JOIN samuel_l_jackson_movies 
                   ON tim_roth_movies.film_name = samuel_l_jackson_movies.film_name''').fetchall()

[('The Hateful Eight', '2015'),
 ('Hardcore Henry', '2015'),
 ("Jumpin' at The Boneyard", '1991'),
 ('Late Night with Seth Meyers', '2016'),
 ('Rotten Tomatoes', '2015'),
 ('Cannes Film Festival', '1998-2015'),
 ('Good Day L.A.', '2013'),
 ('At the Movies', '1998'),
 ('Late Show with David Letterman', '1995'),
 ('The N Word', '2004')]

In [27]:
try:
    engine.execute(''' CREATE TABLE COMMON AS SELECT tim_roth_movies.film_name, tim_roth_movies.year FROM 
                   tim_roth_movies JOIN samuel_l_jackson_movies
                   ON tim_roth_movies.film_name = samuel_l_jackson_movies.film_name''').fetchall()
except:
    pass 

In [28]:
engine.execute('''CREATE TABLE only_tim AS SELECT tim_roth_movies.film_name, tim_roth_movies.year
                                           FROM tim_roth_movies
                                           WHERE tim_roth_movies.film_name NOT IN (SELECT samuel_l_jackson_movies.film_name FROM 
										                                           samuel_l_jackson_movies)''')

In [29]:
engine.execute('''CREATE TABLE only_samuel AS SELECT samuel_l_jackson_movies.film_name, samuel_l_jackson_movies.year
                                           FROM samuel_l_jackson_movies
                                           WHERE samuel_l_jackson_movies.film_name NOT IN (SELECT tim_roth_movies.film_name FROM 
										                                           tim_roth_movies)''')